# Zomato Scraping

In [2]:
#!pip install selenium
import time
import pandas as pd
import numpy as np
import bs4
import selenium
import requests
from selenium import webdriver

In [145]:
import os
os.getcwd()

'C:\\Users\\visha\\Python\\Machine Learning'

In [146]:
browser = webdriver.Chrome('chromedriver.exe')
browser.get('https://www.zomato.com/bangalore/great-food-no-bull')

In [147]:
rest = browser.find_elements_by_css_selector('div.relative.top-res-box.entity-ads-snippet-track')
print(len(rest))

141


In [138]:
df_rest = pd.DataFrame(columns = ['Restaurant Name','URL'])
for item in rest:
    name = item.find_element_by_css_selector('div.res_title.zblack.bold')
    name = name.get_attribute('title')
    url = item.find_element_by_css_selector('a')
    url = url.get_attribute('href')
    curr_row = {
        'Restaurant Name' : name,
        'URL' : url,
    }
    df_rest = df_rest.append(curr_row, ignore_index=True)
df_rest.head()

,Restaurant Name,URL
0,The Terrace at Gilly's Redefined,https://www.zomato.com/bangalore/the-terrace-a...
1,Red Rhino,https://www.zomato.com/bangalore/red-rhino-whi...
2,Burma Burma,https://www.zomato.com/bangalore/burma-burma-i...
3,The Reservoire,https://www.zomato.com/bangalore/the-reservoir...
4,Nasi And Mee,https://www.zomato.com/bangalore/nasi-and-mee-...


In [139]:
Restaurant_Details = pd.DataFrame(columns = ['Restaurant Name',
                                         'Restaurant ID',
                                         'User Name',
                                         'User ID',
                                         'Rating'])

In [141]:
for row in df_rest.values:
    url=row[1]
    browser.get(url)
    
    restname = row[0]
    
    if len(browser.find_elements_by_css_selector('div.clearfix.reviews-subhead-sort')) != 0:
        restid = browser.find_element_by_css_selector('div.res-rating.pos-relative.clearfix.mb5').find_element_by_css_selector('div[data-res-id]').get_attribute('data-res-id')
        print(restname,restid)
        try:
            while(browser.find_element_by_css_selector('div.load-more.bold.ttupper.tac.cursor-pointer.fontsize2') != None):
                browser.find_element_by_css_selector('div.load-more.bold.ttupper.tac.cursor-pointer.fontsize2').click()
                time.sleep(3)
        except:
            users = browser.find_elements_by_css_selector('div.ui.segments.res-review-body.res-review.clearfix.js-activity-root.mbti.item-to-hide-parent.stupendousact')
            for user in users:
                uname = user.find_element_by_css_selector('div.header.nowrap.ui.left').get_attribute('innerText')
                uid = user.find_element_by_css_selector('a[data-entity_id]').get_attribute('data-entity_id')
                rate = user.find_element_by_css_selector('div[aria-label]').get_attribute('aria-label').split(' ')[1]
                #print(restname,restid,uname,uid,rate)
                curr_row = {'Restaurant Name':restname,
                            'Restaurant ID':restid,
                            'User Name':uname,
                            'User ID':uid,
                            'Rating':rate}
                                  
                Restaurant_Details = Restaurant_Details.append(curr_row,ignore_index = True)

    else:
        continue
    
Restaurant_Details.to_csv('Zomato_Reviews.csv',index=False)

The Terrace at Gilly's Redefined 18586207
Red Rhino 18492910
Burma Burma 18625420
The Reservoire 18537585
Nasi And Mee 18476467
O.G. Variar & Sons 55050
You Mee 18802460
The Globe Grub 18526279
Cloud Cafe 18366664
Zee5 Loft 18728203
Punjab Grill 18581089
Bloomsbury's Global Kitchen & Bakehouse 18391020
ECHOES Koramangala 18439634
Shakes Theory 18675459
MISU 18494064
Chinita Real Mexican Food 18608992
The Blue Wagon - Kitchen 18701756
Once Upon A Flame 18160093
Avista Cafe Pizzeria 18628897
BITEBOX 18782299
Float 18423440
Bologna 18682433
Bengaluru Cafe 18630678
Here & Now 18317320
Brewklyn Microbrewery 18426381
The Asian Curry House 18685529
Toast & Tonic 18260063
Bamboo Hut 18431041
Soda Bottle Opener Wala 60673
Veena Stores 52002
Firangi Bake 18614106
Rolls & Bowls 18762699
Caperberry 57730
Sultans of Spice 50943
Buff Buffet Buff 58349
Pasta Street 53497
ANTIGRAVITY 18765119
Nakkshatra  18573016
Pin Me Down 18663873
Suryawanshi 18785458
Namma SLN 18503768
Lavonne 53579
Olive Bar And 

In [142]:
Restaurant_Details.shape

(9500, 5)

In [3]:
eda = pd.read_csv("Zomato_Reviews.csv")

In [4]:
eda.head()

,Restaurant Name,Restaurant ID,User Name,User ID,Rating
0,The Terrace at Gilly's Redefined,18586207,Anusha Stars,1692350,5.0
1,The Terrace at Gilly's Redefined,18586207,Sarkari,2690320,4.0
2,The Terrace at Gilly's Redefined,18586207,Rahul Sali,59657913,5.0
3,The Terrace at Gilly's Redefined,18586207,Priyanka,33555319,5.0
4,The Terrace at Gilly's Redefined,18586207,HungRAI,410091,4.0


# Top 5 Restaurants based on Average Rating

In [41]:
Top_5_Reataurants = eda.groupby('Restaurant Name')['Rating'].mean().sort_values(ascending=False).head()

In [42]:
Top_5_Reataurants

Restaurant Name
Firangi Bake                     4.882353
Shakes Theory                    4.875000
BITEBOX                          4.827586
Karavalli - The Gateway Hotel    4.764706
Punjab Grill                     4.669065
Name: Rating, dtype: float64

# Top 5 Restaurants based on no. of ratings

In [45]:
eda.groupby('Restaurant Name')['User ID'].count().sort_values(ascending=False).head()

Restaurant Name
The Reservoire                            1487
Burma Burma                                734
Nasi And Mee                               515
The Pizza Bakery                           210
Brew and Barbeque - A Microbrewery Pub     206
Name: User ID, dtype: int64

# Top 5 Restaurants based on Score

In [50]:
Average_Score_DF = pd.DataFrame(eda.groupby('Restaurant Name')['Rating'].mean())
Average_Score_DF['Count'] = (eda.groupby('Restaurant Name')['User ID'].count())
Average_Score_DF['Score'] = Average_Score_DF['Rating']*Average_Score_DF['Count']

In [53]:
Average_Score_DF.sort_values('Score',ascending=False).head()

,Rating,Count,Score
Restaurant Name,,,
The Reservoire,4.405178,1487,6550.5
Burma Burma,4.278610,734,3140.5
Nasi And Mee,4.097087,515,2110.0
The Pizza Bakery,4.509524,210,947.0
Brew and Barbeque - A Microbrewery Pub,4.138350,206,852.5


# Least 5 Restaurants based on Average Rating

In [54]:
eda.groupby('Restaurant Name')['Rating'].mean().sort_values().head()

Restaurant Name
Prost Brew Pub              3.79798
Imperio Restaurant          3.80000
Brooks and Bonds Brewery    3.86500
Empire Restaurant           4.00000
One For The Road            4.00000
Name: Rating, dtype: float64

# Top 5 Users

In [56]:
eda.groupby('User Name')['Rating'].count().sort_values(ascending=False).head()

User Name
Vineel Chandra    19
Eshwar Vemala     17
Diptarka Gupta    17
_poodforn         15
Sunil Kumar       15
Name: Rating, dtype: int64

# Recommendation Engine

In [57]:
rest = pd.read_csv("Zomato_Reviews.csv")

In [58]:
rest.head()

,Restaurant Name,Restaurant ID,User Name,User ID,Rating
0,The Terrace at Gilly's Redefined,18586207,Anusha Stars,1692350,5.0
1,The Terrace at Gilly's Redefined,18586207,Sarkari,2690320,4.0
2,The Terrace at Gilly's Redefined,18586207,Rahul Sali,59657913,5.0
3,The Terrace at Gilly's Redefined,18586207,Priyanka,33555319,5.0
4,The Terrace at Gilly's Redefined,18586207,HungRAI,410091,4.0


In [61]:
rest['User ID'].nunique(),rest['Restaurant ID'].nunique()

(5341, 112)

In [62]:
ui_matrix = rest.pivot_table(index = 'User ID',
                               columns = 'Restaurant ID',
                               values = 'Rating') 
ui_matrix.head()

Restaurant ID,50462,50471,50565,50663,50742,50857,50879,50943,51705,52002,...,18767303,18767897,18772498,18782299,18783527,18785458,18785530,18795951,18801352,18802460
User ID,,,,,,,,,,,,,,,,,,,,,
238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6719,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN


In [63]:
ui_matrix_norm = ui_matrix.apply(lambda v: (v - v.mean())/v.std(), axis=1).fillna(0)

In [65]:
ui_matrix_norm.head(10)

Restaurant ID,50462,50471,50565,50663,50742,50857,50879,50943,51705,52002,...,18767303,18767897,18772498,18782299,18783527,18785458,18785530,18795951,18801352,18802460
User ID,,,,,,,,,,,,,,,,,,,,,
238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
6686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
6719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
7693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
8433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
8852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.57735
9082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
9935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
11876,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000


In [66]:
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
user_sim = pd.DataFrame(cosine_similarity(ui_matrix_norm),
                       index = ui_matrix_norm.index,
                       columns = ui_matrix_norm.index)
user_sim.head()

User ID,238,6686,6719,7693,8433,8852,9082,9935,11876,13486,...,83286219,83298879,85051219,85438430,86296973,86328912,87017594,87384749,88648927,89153448
User ID,,,,,,,,,,,,,,,,,,,,,
238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7693,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
userid = 6719
neighbours = user_sim[userid].drop(userid).sort_values(ascending = False).head(3)
neighbours = neighbours.index
neighbours

Int64Index([89153448, 2759078, 2781008], dtype='int64', name='User ID')

In [104]:
user_rest = ui_matrix.loc[userid]
rest_not_visited = user_rest[pd.isnull(user_rest)]

In [105]:
neighs_rest_matrix = ui_matrix_norm.loc[neighbours]
neighs_rest_matrix

rest_predictions = neighs_rest_matrix[rest_not_visited.index].mean()
rest_recomended = rest_predictions.sort_values(ascending=False).head()
rest_recomended = rest_recomended.index
rest_recomended

Int64Index([55419, 18802460, 58268, 58818, 60673], dtype='int64', name='Restaurant ID')

In [108]:
rest[rest['Restaurant ID'].isin(rest_recomended)]['Restaurant Name'].unique()

array(['You Mee', 'Soda Bottle Opener Wala', 'Yauatcha', 'BelgYum',
       'The Fatty Bao - Asian Gastro Bar'], dtype=object)